In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Training utils
from training_code.utils import utils_xgboost

# Optuna
import optuna
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# VALIDACION

In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])
validacion = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

In [3]:
e1 = validacion[validacion['ESTACION'] == 1]

e1.FECHAHORA.min()

Timestamp('2021-03-30 13:05:00')

In [14]:
%%time

predicciones = {}
metricas = {}

for i in range(1, 11):
    
    estacion = i

    variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 
                 'TEMPERATURA', 'HUMEDAD', 'PRESION', 'TEMPERATURA_PRONOSTICO', 
                 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO', 'DIA_SEM', 'TRAFICO' , 'AQI_MP10', 'AQI_MP2_5']

    dependent = ['AQI_MP2_5']

    number_of_features = len(variables)

    training_days = 7 
    forecast_days = 1/2
    samples_per_day = 288
    step = 288/2

    # Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
    train_months = relativedelta(months = 12)
    test_months = relativedelta(months = 2, days = 20)

    input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
    output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
    train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(datos, 
                                                                    estacion,
                                                                    train_months, 
                                                                    variables, 
                                                                    dependent, 
                                                                    train_test_samples, 
                                                                    input_samples, 
                                                                    output_samples, 
                                                                    number_of_features,
                                                                    step)

    X_train_val, y_train_val = utils_xgboost.get_validation(validacion, 
                                                                    estacion,
                                                                    variables, 
                                                                    dependent, 
                                                                    train_test_samples, 
                                                                    input_samples, 
                                                                    output_samples, 
                                                                    number_of_features,
                                                                    step)

    params = {'max_depth': 6, 
    'learning_rate': 0.016170340622682584, 
    'n_estimators': 282, 
    'min_child_weight': 10, 
    'gamma': 0.006843610407559761, 
    'subsample': 0.3168966517747982, 
    'colsample_bytree': 0.6780093701705895}

    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_train_val)
    
    # guardamos los valores predecidos vs reales en un diccionario
    
    predicciones[i] = {'real' : y_train_val, 'prediccion': prediction}


    pickle.dump(trained_xgb_model, open('models/models_xgboost/NUEVO/7_dias_STEP_CORRECTO/xgboost_12hs_estacion_' + str(i) + '.pkl', 'wb'))

    mean_real = y_train_val.mean()
    mean_prediction = prediction.mean()

    MAPE = mean_absolute_percentage_error(prediction, y_train_val)
    MAE = mean_absolute_error(prediction, y_train_val)
    RMSE = mean_squared_error(prediction, y_train_val, squared = False)
    
    # guardamos las metricas en un diccionario
    
    metricas[i] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}
    
    print('ESTACION '+ str(i) + ':')
    print('prediction shape: ', prediction.shape)
    print('test shape: ', y_train_val.shape)
    print('MAE :', MAE)
    print('MAPE: ', MAPE)
    print('RMSE: ', RMSE)
    print('\n')
    print('media real: ', mean_real)
    print('media predecida: ', mean_prediction)
    print('\n')


ESTACION 1:
prediction shape:  (170, 144)
test shape:  (170, 144)
MAE : 5.3284153919983535
MAPE:  0.17039117748883958
RMSE:  7.987222922280712


media real:  37.459722222222226
media predecida:  37.41293


ESTACION 2:
prediction shape:  (170, 144)
test shape:  (170, 144)
MAE : 4.246327663870419
MAPE:  0.13304503342455495
RMSE:  6.1602974818586995


media real:  34.08039215686274
media predecida:  32.43762


ESTACION 3:
prediction shape:  (170, 144)
test shape:  (170, 144)
MAE : 5.1105225457864645
MAPE:  0.11141011440746557
RMSE:  7.842876822895799


media real:  46.45600490196078
media predecida:  46.326992


ESTACION 4:
prediction shape:  (170, 144)
test shape:  (170, 144)
MAE : 5.264705452108695
MAPE:  0.16060186657150777
RMSE:  7.895815965195131


media real:  35.51466503267974
media predecida:  34.163364


ESTACION 5:
prediction shape:  (170, 144)
test shape:  (170, 144)
MAE : 4.639979950859656
MAPE:  0.14160661870517005
RMSE:  8.326777659693597


media real:  32.33815359477124
med

In [15]:
total = 0

for i in [1,2,3,4,5,6,7,8,9,10]:
    total = total + metricas[i]['MAPE']

print(total/9)

total = 0

for i in [1,2,3,4,5,6,7,8,9,10]:
    total = total + metricas[i]['MAE']

print(total/9)
total = 0
    
for i in [1,2,3,4,5,6,7,8,9,10]:
    total = total + metricas[i]['RMSE']

print(total/9)

total = 0
    
for i in [1,2,3,4,5,6,7,8,9,10]:
    total = total + metricas[i]['Media real']

print(total/9)

total = 0
    
for i in [1,2,3,4,5,6,7,8,9,10]:
    total = total + metricas[i]['Media predecida']

print(total/10)

0.17660512152854269
6.137354509905792
10.731458443268322
40.51754720406681
34.80525550842285


# sin 9

MAPE: 0.21530257027753344

MAE: 7.993908123285844

RMSE: 12.022785664353094

MEDIA REAL: 34.650304978569075

MEDIA PREDECIDA: 37.26750183105469

0.25308908786679635
9.347597419453775
14.11784873672186
38.433946078431376
36.96553103129069


# con 9


MAPE: 0.2525008354814956

MAE: 8.705785048478385

RMSE: 12.957564849084738

MEDIA REAL: 36.50731639465876

MEDIA PREDECIDA: 35.6290786743164



0.26186168330982657
9.057862853055296
13.494228552565975
36.46579248366013
35.448756790161134




In [16]:
df_metricas = pd.DataFrame.from_dict(metricas)

df_metricas.to_csv('metrics/XGBOOST/metricas_10estaciones_7dias_mape_step12hs.csv')

In [17]:
predicciones[1]['real'].flatten()

array([25., 25., 25., ..., 66., 66., 66.])

In [18]:
list_dfs = []

for i in range(1,11):
    d = {'TARGET': predicciones[i]['real'].flatten(), 'FORECAST': predicciones[i]['prediccion'].flatten()}
    df_aux = pd.DataFrame(data = d)
    df_aux['ESTACION'] = i
    list_dfs.append(df_aux)

df_predicciones = pd.concat(list_dfs)

df_predicciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244800 entries, 0 to 24479
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   TARGET    244800 non-null  float64
 1   FORECAST  244800 non-null  float32
 2   ESTACION  244800 non-null  int64  
dtypes: float32(1), float64(1), int64(1)
memory usage: 6.5 MB


In [19]:
df_predicciones.to_csv('datos/predicciones_10estaciones_7dias_step12hs.csv')

In [20]:
for i in range(1,11):

    fig_val = go.Figure()

    df_grafica = df_predicciones[df_predicciones['ESTACION'] == i]

    fig_val.add_trace(
        go.Scatter( y = list(df_grafica.TARGET), name = 'Target'))

    fig_val.add_trace(
        go.Scatter( y = list(df_grafica.FORECAST), name = 'Forecasts'))

    fig_val.update_layout( title_text = "Validation - Forecasts vs Targets")

    fig_val.write_html('graphs/XGBOOST/XGBOOST_12hs_estacion_'+str(i)+'_targets_vs_forecasts.html')

In [37]:



%%time


# Hiperparametros Optuna parado

# params = {'max_depth': 6, 
#           'learning_rate': 0.010049185067138871, 
#           'n_estimators': 294, 
#           'min_child_weight': 6, 
#           'gamma': 0.0012980577270314173, 
#           'subsample': 0.18828660906502742, 
#           'colsample_bytree': 0.9488588949410688}

#Hiperparametros ejemplo

# params = {'learning_rate' : 0.025,
#           'n_estimators' : 250,
#           'max_depth': 2,
#           'min_child_weight' : 1,
#           'gamma': 0.0,
#           'subsample': 0.98,
#           'colsample_bytree': 0.98,
#           'scale_pos_weight': 0.8,
#           'seed': 42,
#           'verbosity' : 0}


# Hiperparametros optuna completado



prediction (213, 288)
test (213, 288)
RMSE:  10.866205543857895
MAE:  7.2546269617465615
CPU times: total: 1h 9min 33s
Wall time: 6min 31s


28.150202138758477
27.673008


0.27459897048248555

# Valores para parametros de optuna incompleto

* media real: 28.150202138758477
* media predecida: 27.714634


* RMSE:  10.886802850195714
* MAE:  7.321197804384202
* MAPE: 0.2760916690053607

# Valores para parametros de optuna terminado

* 28.150202138758477
* 27.673008

* RMSE:  10.866205543857895
* MAE:  7.2546269617465615
* MAPE: 0.27459897048248555

# Valores para parametros del paper

* Media real: 28.150202138758477
* Media predecida: 29.366629

* RMSE:  10.724885901102555
* MAE:  7.297981035549254

* MAPE: 0.2604501639385927

# Arreglos para graficar

In [12]:
pred_1 = []


prediction = predicciones[4]['prediccion']
y_real = predicciones[4]['real']


for i in range(3, len(prediction), 4):
    pred_1.append(prediction[i])
    
pred_1 = np.asarray(pred_1)
    
test_1 = []

for i in range(3, len(y_real), 4):
    test_1.append(y_real[i])
    
test_1 = np.asarray(test_1)

print(pred_1.shape)
print(test_1.shape)

(84, 288)
(84, 288)


In [13]:
y_pred = np.reshape(pred_1, ( len(pred_1) * len(pred_1[0])))
y_test_1 = np.reshape(test_1, (len(test_1) * len(test_1[0])))

print(y_pred)
print(y_test_1)

fig_val = go.Figure()

fig_val.add_trace(
    go.Scatter( y = list(y_test_1), name = 'Target'))

fig_val.add_trace(
     go.Scatter( y = list(y_pred), name = 'Forecasts'))

fig_val.update_layout( title_text = "Validation - Forecasts vs Targets")

[ 7.348355  7.580247  7.677126 ... 28.63128  30.697857 29.67563 ]
[ 4.  4.  4. ... 29. 29. 29.]


In [14]:
for i in range(1,11):

    prediction = predicciones[i]['prediccion']
    y_val = predicciones[i]['real']

    y_val.shape

    pred_1 = []

    for j in range(1, len(prediction), 4):
        pred_1.append(prediction[j])
        
    pred_1 = np.asarray(pred_1)
        
    test_1 = []

    for j in range(1, len(y_val), 4):
        test_1.append(y_val[j])
        
    test_1 = np.asarray(test_1)

    print(pred_1.shape)
    print(test_1.shape)

    y_pred = np.reshape(pred_1, ( len(pred_1) * len(pred_1[0])))
    y_test_1 = np.reshape(test_1, (len(test_1) * len(test_1[0])))

    print(y_pred)
    print(y_test_1)

    fig_val = go.Figure()

    fig_val.add_trace(
        go.Scatter( y = list(y_test_1), name = 'Valor Real'))

    fig_val.add_trace(
        go.Scatter( y = list(y_pred), name = 'Valor Predecido'))

    fig_val.update_layout( title_text = "Validación - Estación " + str(i)+ " - Forecasts vs Targets")

    fig_val.write_html('graphs/XGBOOST/XGBOOST_NUEVO_estacion_'+str(i)+'_targets_vs_forecasts.html')

(84, 288)
(84, 288)
[20.638203 20.639692 20.794905 ... 40.957756 44.665794 41.97433 ]
[21. 21. 21. ... 25. 25. 25.]
(84, 288)
(84, 288)
[ 7.9361377  8.025823   8.144635  ... 33.357754  33.71797   34.51716  ]
[ 8.  8.  8. ... 25. 25. 25.]
(84, 288)
(84, 288)
[32.6527   32.62108  32.626762 ... 34.267822 36.012054 35.78499 ]
[33. 33. 33. ... 38. 38. 38.]
(84, 288)
(84, 288)
[41.842545 41.84881  41.476147 ... 24.163519 25.307867 24.63339 ]
[42. 42. 42. ... 29. 29. 29.]
(84, 288)
(84, 288)
[11.982722 11.950415 11.949314 ... 26.18529  27.92822  27.082037]
[12. 12. 12. ... 21. 21. 21.]
(84, 288)
(84, 288)
[28.918186 29.031517 29.18205  ... 44.876568 43.356304 43.260017]
[29. 29. 33. ... 42. 42. 42.]
(84, 288)
(84, 288)
[11.966377 11.966511 12.181568 ... 24.796286 23.86486  24.526014]
[12. 12. 12. ... 21. 21. 21.]
(84, 288)
(84, 288)
[24.792011 24.823029 24.83603  ... 39.99317  39.360428 39.152203]
[25. 25. 25. ... 33. 33. 33.]
(84, 288)
(84, 288)
[68.34663  67.651054 67.81925  ... 24.748415 2